In [ ]:
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
import os
import pandas as pd
import uuid
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict, Any
from IPython.display import Image, display
import pprint
import time

In [57]:
# --- Helper Models for Quiz Variants ---
class FlashCardData(BaseModel):
    word_hanzi: str
    pinyin: str
    definition_en: str
    definition_vi: str
    example_sentence: Optional[str] = None
    example_pinyin: Optional[str] = None
    example_meaning_en: Optional[str] = None
    example_meaning_vi: Optional[str] = None
    extra_info_md: Optional[str] = None

class QuizOptionData(BaseModel):
    """Used for Single Choice, Multiple Choice, Listening, Image Selection"""
    text: str = Field(..., description="The answer text or label")
    is_correct: bool
    order: int
    image_url: Optional[str] = Field(None, description="URL if the option is an image")

class MatchPairData(BaseModel):
    """Used for Match Pairs"""
    left_text: str
    right_text: str
    order: int

class SentenceQuizData(BaseModel):
    """Used for Fill in the Blank, Order Sentence"""
    chinese_sentence: str
    pinyin_sentence: Optional[str] = None
    correct_answer: Optional[str] = Field(None, description="The specific word for 'Fill in the Blank'")
    distractors: List[str] = Field(default_factory=list, description="Wrong word blocks")

class TextResponseQuizData(BaseModel):
    """Used for Translate (CN<->EN/VI) and Writing"""
    source_text: Optional[str] = Field(None, description="Text to translate or prompt for writing")
    accepted_answers: List[str] = Field(default_factory=list, description="Valid variations of the translation")
    is_strict: bool = False

class SpeakingQuizData(BaseModel):
    """Used for Speaking"""
    expected_text: str
    expected_pinyin: Optional[str] = None
    time_limit: int = 15

# --- Core Models ---

class GeneratedQuiz(BaseModel):
    quiz_type: Literal[
        'Flashcard', 
        'Single Choice', 'Multiple Choice', 'Image Selection', 
        'Fill in the Blank', 'Translate to CN', 'Translate from CN', 'Writing',
        'Listening', 'Speaking', 
        'Match Pairs', 'Order Sentence'
    ]
    
    question_en: str
    question_vi: str
    explanation_en: Optional[str] = None
    explanation_vi: Optional[str] = None
    
    # Media prompts at the Quiz level
    audio_url: Optional[str] = None
    image_url: Optional[str] = None
    
    order: int
    
    # Polymorphic fields: Map the Enum type to the correct data container
    # 1. Flashcard
    flashcard: Optional[FlashCardData] = None
    
    # 2. Options (Single Choice, Multiple Choice, Image Selection, Listening)
    options: Optional[List[QuizOptionData]] = None
    
    # 3. Pairs (Match Pairs)
    pairs: Optional[List[MatchPairData]] = None
    
    # 4. Sentence (Fill in the Blank, Order Sentence)
    sentence: Optional[SentenceQuizData] = None
    
    # 5. Text Input (Translate..., Writing)
    text_response: Optional[TextResponseQuizData] = None
    
    # 6. Speaking (Speaking)
    speaking: Optional[SpeakingQuizData] = None

class GeneratedSection(BaseModel):
    # STRICTLY MATCHING YOUR LESSON SECTION TYPE ENUM
    section_type: Literal[
        'Vocabulary', 'Grammar', 'Reading', 'Listening', 
        'Speaking', 'Conversation', 'Final Test'
    ]
    title_en: str
    title_vi: str
    content_md: Optional[str] = Field(None, description="Markdown theory/dialogue text")
    order: int
    quizzes: List[GeneratedQuiz] = Field(..., description="List of quizzes in this section")

class GeneratedLessonContent(BaseModel):
    sections: List[GeneratedSection]
    
class CritiqueEvaluation(BaseModel):
    approved: bool = Field(description="Set to True if the content is perfect for the HSK level. False if changes are needed.")
    feedback: str = Field(description="Specific instructions on what to fix (e.g., 'The word 'Economy' is too hard for HSK1', 'Distractors in Q3 are too obvious').")
    
class SectionBlueprint(BaseModel):
    """The Architect's plan for a single section."""
    section_type: Literal['Vocabulary', 'Grammar', 'Reading', 'Listening', 'Speaking', 'Conversation', 'Final Test']
    title_en: str
    title_vi: str
    teaching_goal: str = Field(..., description="Pedagogical goal (e.g., 'Introduce basic greetings').")
    required_concepts: List[str] = Field(..., description="Specific words or grammar points to cover in this section.")
    recommended_quiz_types: List[str] = Field(..., description="Which quiz types best fit this section (e.g., 'Flashcard', 'Match Pairs').")

class LessonBlueprint(BaseModel):
    """Output of Agent 1"""
    rationale: str = Field(..., description="Reasoning for this lesson's structure.")
    sections: List[SectionBlueprint]

In [ ]:
# --- Setup LLMs ---
planner_llm = ChatOpenAI(
    model="gpt-4o",  
    temperature=0.3,
    streaming=True,
    api_key=""
) 

# WRITER: High creativity, high volume (runs in loop).
writer_llm = ChatOpenAI(
    model="gpt-4o", # Stick to 4o for complex JSON. Use 'gpt-4o-mini' ONLY if budget is tight.
    temperature=0.7, # Higher temp for diverse, natural sentences
    streaming=True,
    api_key=""
)

# REVIEWER: Strict logic, medium volume.
critic_llm = ChatOpenAI(
    model="gpt-4o", 
    temperature=0.0, # Zero temp for maximum strictness and consistency
    streaming=True,
    api_key=""
)

# --- Define State ---
class AgentState(TypedDict):
    # Inputs
    course_info: dict
    lesson_info: dict
    
    # Internal State
    lesson_plan: Optional[LessonBlueprint]       # Output of Agent 1
    draft_content: Optional[GeneratedLessonContent] # Output of Agent 2
    
    # Feedback Loop
    critique_feedback: Optional[str]             # Output of Agent 3
    revision_count: int 
    is_approved: bool


In [66]:
class PlannerState(TypedDict):
    course_info: dict
    lesson_info: dict
    lesson_plan: Optional[LessonBlueprint]
# --- 3. AGENT 1: The Architect (Planner) ---
def architect_node(state: PlannerState):
    print(f"\n🔵 [ARCHITECT] Planning structure...")
    course = state['course_info']
    lesson = state['lesson_info']
    parser = PydanticOutputParser(pydantic_object=LessonBlueprint)
    
    system_msg = """
    You are a Senior HSK Curriculum Architect. 
    Your job is to PLAN the structure of a lesson. Do not write the quizzes yet.
    
    ### STRICT ENUM RULES
    You must ONLY use the following `section_type` values:
    - 'Vocabulary' (For Flashcards)
    - 'Grammar' (For Theory)
    - 'Reading' (For Text comprehension practice)
    - 'Listening' (For Audio comprehension practice)
    - 'Speaking' (For Pronunciation)
    - 'Conversation' (For Dialogue)
    - 'Final Test' (For General Review/Mixed Practice)
    
    **CRITICAL:** Do NOT use "Practice" as a section type. If you want a practice section, use 'Reading', 'Listening', or 'Final Test'.

    ### INSTRUCTIONS
    1. Analyze the Learning Objectives.
    2. Must define 7-8 Sections (Scaffolding: Acquisition -> Application -> Production).
    3. Explain WHY each section is needed in the `rationale`.
    4. List specific concepts (words/grammar) required for each section.
    """
    
    user_msg = "Course: {course_name} ({hsk_level})\nLesson: {lesson_title}\nObjectives: {objectives}"
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("user", user_msg),
        ("user", "\nOutput Blueprint JSON:\n{format_instructions}")
    ])
    
    formatted_inputs = {
        "course_name": course['name'],
        "hsk_level": course['hsk_level'],
        "lesson_title": lesson['title'],
        "objectives": lesson['learning_objectives'],
        "format_instructions": parser.get_format_instructions()
    }
    
    # Add error handling to retry if it fails (optional but recommended)
    try:
        response = planner_llm.invoke(prompt.invoke(formatted_inputs))
        plan = parser.parse(response.content)
        print(f"\n\n🔎 [ARCHITECT OUTPUT]:\n{plan.model_dump_json(indent=2)}")
        return {"lesson_plan": plan}
    except Exception as e:
        # If the Architect fails, we can't proceed. In a production app, you might loop here.
        # For now, we return empty which will stop the graph gracefully or throw.
        print(f"Architect Error: {e}") 
        raise e
    
planner_workflow = StateGraph(PlannerState)
planner_workflow.add_node("architect", architect_node)
planner_workflow.set_entry_point("architect")
planner_workflow.add_edge("architect", END)
planning_graph = planner_workflow.compile()

In [68]:
class GeneratorState(TypedDict):
    # Inputs
    course_info: dict
    section_plan: SectionBlueprint # We only input ONE section plan here
    
    # Outputs
    draft_section: Optional[GeneratedSection] # We output ONE section content
    
    # Loop controls
    critique_feedback: Optional[str]
    revision_count: int
    is_approved: bool

# --- AGENT 2: The Content Creator (Writer) ---
def writer_node(state: GeneratorState):
    # Inputs
    plan = state['section_plan']
    feedback = state.get('critique_feedback')
    revisions = state.get('revision_count', 0)
    
    print(f"\n🟡 [WRITER] Generating Section: {plan.title_en} (Rev {revisions})...")
    
    parser = PydanticOutputParser(pydantic_object=GeneratedSection)
    
    system_msg = """
    You are an Expert Content Writer for a Chinese Learning App (HSK Standard).
    Your goal is to turn a high-level Lesson Blueprint into concrete JSON data.

    ### GENERAL GUIDELINES
    1. **Strict Adherence:** Must follow the `lesson_plan` exactly. If the plan asks for "Order Sentence" for "你好", do not create a "Flashcard".
    2. **Language Level:** Use *only* vocabulary appropriate for the target HSK level.
    3. **Bilingual:** All explanations/definitions must be in both English (`_en`) and Vietnamese (`_vi`).
    4. **Quantity:** Must have at least 18-20 quizzes for a section.

    ### QUIZ TYPE SPECIFICATIONS (STRICT RULES)

    **1. 'Flashcard' (Vocabulary Only)**
       - **Usage:** Introducing new words.
       - **Data:** Fill `flashcard` field.
       - **Requirements:** - `word_hanzi`: The character(s).
         - `pinyin`: The pronunciation in Pinyin.
         - `definition_en`: Concise English meaning.
         - `definition_vi`: Concise Vietnamese meaning.
         - `example_sentence`: A simple sentence (mostly known words + this new word).
         - `example_pinyin`: Pinyin for the example sentence.
         - `example_meaning_en`: English meaning of the example sentence.
         - `example_meaning_vi`: Vietnamese meaning of the example sentence.
         - `extra_info_md`: Optional extra info in Markdown (e.g., cultural notes).

    **2. 'Single Choice' (Grammar/Reading)**
       - **Usage:** Standard testing.
       - **Data:** Fill `options` list.
       - **Structure:** Exactly **4 Options**.
       - **Logic:** 1 Option has `is_correct=True`, 3 Options have `is_correct=False`.
       - **Distractors:** Wrong answers must be grammatically plausible or visual lookalikes.

    **3. 'Multiple Choice' (Select All)**
       - **Usage:** When multiple answers are valid.
       - **Data:** Fill `options` list.
       - **Logic:** At least 2 Options must be `is_correct=True`.

    **4. 'Match Pairs' (Gamification)**
       - **Usage:** Testing Vocabulary recall.
       - **Data:** Fill `pairs` list.
       - **Structure:** Generate exactly at least **4 pairs** per quiz.
       - **Format:** `left_text` = Hanzi, `right_text` = Pinyin/Meaning.

    **5. 'Order Sentence' (Syntax Practice)**
       - **Usage:** Grammar practice.
       - **Data:** Fill `sentence` field.
       - **Logic:** Provide the **Full Correct Sentence** in `chinese_sentence`.
       - **Note:** The Frontend will handle the shuffling. You just provide the correct answer.

    **6. 'Fill in the Blank' (Context Practice)**
       - **Usage:** Testing specific vocabulary in context.
       - **Data:** Fill `sentence` field.
       - **Logic:** - `chinese_sentence`: The full sentence *including* the word to be removed.
         - `correct_answer`: The specific word that goes in the blank.
         - `distractors`: List of 3 incorrect words that fit grammatically but not semantically.

    **7. 'Listening' (Comprehension)**
       - **Usage:** Audio testing.
       - **Data:** Fill `options` list.
       - **Content:** The `question_en` should be "Listen and select the correct answer" or "What did you hear?".
       - **Note:** Leave `audio_url` empty (None) unless you have a specific placeholder URL strategy.

    **8. 'Speaking' (Pronunciation)**
       - **Usage:** User reads text aloud.
       - **Data:** Fill `speaking` field.
       - **Logic:** `expected_text` is the Hanzi the user must say. `time_limit` usually 10-15s.

    **9. 'Translate to CN' / 'Writing'**
       - **Usage:** Output/Production practice.
       - **Data:** Fill `text_response` field.
       - **Logic:** `accepted_answers` must be a list of valid string variations (e.g. ["Hello", "Hi"]).

    ### MUST HAVE:
    - **Quantity:** Must have at least 18-20 quizzes for a section lesson(Around 126 ~ 144 quizzes for a full lessons).
    
    ### FORMATTING
    Output strict JSON matching the schema. Do not explain your code.
    """
    
    user_msg_template = """
    BLUEPRINT:
    {blueprint_json}
    """
    
    if feedback and revisions > 0:
        user_msg_template += "\n\n!!! CRITICAL FEEDBACK !!!\nReviewer Feedback: {feedback}\nFix issues and regenerate."

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("user", user_msg_template),
        ("user", "\nOutput Final Content JSON:\n{format_instructions}")
    ])
    
    # Prepare inputs safely
    inputs = {
        "blueprint_json": plan.model_dump_json(), # Pass JSON string as variable content
        "format_instructions": parser.get_format_instructions(),
        "feedback": feedback if feedback else ""
    }
    try:
        response = writer_llm.invoke(prompt.invoke(inputs))
        draft = parser.parse(response.content)
        print(f"\n\n📝 [WRITER OUTPUT]:\n{draft.model_dump_json(indent=2)}")
        return {
            "draft_section": draft,
            "revision_count": revisions + 1
        }
    except Exception as e:
        return {"critique_feedback": f"JSON Parsing Error: {str(e)}", "revision_count": revisions + 1}

In [79]:
# --- 5. AGENT 3: The Reviewer (Birdbrain) ---
def reviewer_node(state: GeneratorState):
    print(f"\n🔴 [REVIEWER] Checking {state['section_plan'].title_en}...")
    draft = state.get('draft_section')
    
    if not draft:
        print("   ⚠️ No draft found (Writer failed). Rejecting automatically.")
        return {
            "is_approved": False, 
            "critique_feedback": "Writer failed to generate valid JSON. Please retry."
        }

    course = state['course_info']
    parser = PydanticOutputParser(pydantic_object=CritiqueEvaluation)
        
    system_msg = f"""
    You are 'Birdbrain', a strict QA Auditor.
    
    CHECKLIST:
    1. Distractor Quality: Are wrong answers plausible?
    2. Naturalness: Is the Chinese natural?
    3. Alignment: Does content match the Architect's Blueprint?
    
    Return 'approved': True ONLY if perfect.
    """
    
    user_msg_template = """
    DRAFT TO REVIEW:
    {draft_json}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("user", user_msg_template),
        ("user", "{format_instructions}")
    ])
    
    response = critic_llm.invoke(prompt.invoke({
        "draft_json": draft.model_dump_json(), 
        "format_instructions": parser.get_format_instructions()
    }))
    eval_result = parser.parse(response.content)
    print(f"\n\n⚖️ [REVIEWER OUTPUT]:\n{eval_result.model_dump_json(indent=2)}")
    return {"is_approved": eval_result.approved, "critique_feedback": eval_result.feedback}

In [80]:
def should_continue(state: GeneratorState):
    if state['is_approved']: return "end"
    if state['revision_count'] > 3: return "end"
    print(f"❌ Rejected. Looping...")
    return "revise"

In [81]:
gen_workflow = StateGraph(GeneratorState)
gen_workflow.add_node("writer", writer_node)
gen_workflow.add_node("reviewer", reviewer_node)
gen_workflow.set_entry_point("writer")
gen_workflow.add_edge("writer", "reviewer")
gen_workflow.add_conditional_edges("reviewer", should_continue, {"end": END, "revise": "writer"})
generation_graph = gen_workflow.compile()

In [45]:
#Get the graph visualization in Mermaid format
print(generation_graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	writer(writer)
	reviewer(reviewer)
	__end__([<p>__end__</p>]):::last
	__start__ --> writer;
	reviewer -. &nbsp;end&nbsp; .-> __end__;
	reviewer -. &nbsp;revise&nbsp; .-> writer;
	writer --> reviewer;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [83]:
def main():
    # 1. Setup & Config
    if not os.path.exists('../input_data/lessons.csv'):
        print("❌ Error: Input files not found.")
        return

    courses_df = pd.read_csv('../input_data/courses.csv') 
    lessons_df = pd.read_csv('../input_data/lessons.csv')
    
    OUTPUT_DIR = 'output_data'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 2. Global Counters
    section_id_counter = 1
    quiz_id_counter = 1

    print(f"\n🚀 Starting Pipeline (Force Save Mode)...\n")

    for _, lesson_row in lessons_df.iterrows():
        print(f"\n╔══════════════════════════════════════════════════════════════╗")
        print(f"║ Processing Lesson: {lesson_row['title']}")
        print(f"╚══════════════════════════════════════════════════════════════╝")
        
        course_row = courses_df[courses_df['id'] == lesson_row['course']].iloc[0]

        # --- PHASE 1: PLAN ---
        blueprint = None
        while not blueprint:
            try:
                print("   [1/2] 🏗️  Architect is designing...")
                plan_state = planning_graph.invoke({
                    "course_info": course_row.to_dict(),
                    "lesson_info": lesson_row.to_dict(),
                    "lesson_plan": None
                })
                blueprint = plan_state['lesson_plan']
                print(f"   ✅  Blueprint ready: {len(blueprint.sections)} sections.")
            except Exception as e:
                print(f"   ❌ Architect Error: {e}. Retrying...")
                time.sleep(2)

        # --- PHASE 2: GENERATE SECTIONS ---
        lesson_id = lesson_row['id']
        
        for index, section_plan in enumerate(blueprint.sections):
            print(f"\n   [2/2] 🏭 Section {index + 1}/{len(blueprint.sections)}: {section_plan.section_type}")
            
            generated_section = None
            attempt = 1
            max_retries = 3 # Only retry if the Writer crashes completely (produces None)
            
            # --- GENERATION LOOP ---
            while not generated_section and attempt <= max_retries:
                try:
                    gen_state = generation_graph.invoke({
                        "course_info": course_row.to_dict(),
                        "section_plan": section_plan,
                        "draft_section": None,
                        "critique_feedback": None,
                        "revision_count": 0,
                        "is_approved": False
                    })
                    
                    draft = gen_state.get('draft_section')
                    approved = gen_state.get('is_approved')
                    revisions = gen_state.get('revision_count')

                    # --- LOGIC CHANGE HERE ---
                    if draft:
                        # We have content!
                        if approved:
                            print(f"      ✅  Approved!")
                        else:
                            # If not approved but we have a draft, we FORCE SAVE it
                            print(f"      ⚠️  Max Revisions ({revisions}) reached. Force saving unapproved draft.")
                        
                        generated_section = draft # Accept the draft to break the loop
                    else:
                        # Draft is None (Writer failed parsing JSON completely)
                        print(f"      ❌  Writer produced Nothing (JSON Error). Retrying (Attempt {attempt})...")
                        attempt += 1
                        
                except Exception as e:
                    print(f"      ❌ Graph Crash: {e}. Retrying...")
                    time.sleep(1)
                    attempt += 1

            # Check if we failed completely after retries
            if not generated_section:
                print(f"      💀 SKIPPING Section: Could not generate valid JSON after {max_retries} attempts.")
                continue # Skip to next section

            # --- PHASE 3: SAVE ---
            # (Same saving logic as before)
            local_sections = []
            local_quizzes = []
            local_flashcards = []
            local_options = []
            local_pairs = []
            local_sentences = []
            local_speaking = []
            local_text_response = []

            current_section_id = section_id_counter
            section_id_counter += 1
            
            local_sections.append({
                "id": current_section_id, 
                "lesson": lesson_id, 
                "section_type": generated_section.section_type, 
                "title_en": generated_section.title_en, 
                "title_vi": generated_section.title_vi, 
                "content_md": generated_section.content_md, 
                "order": index + 1
            })

            for quiz in generated_section.quizzes:
                current_quiz_id = quiz_id_counter
                quiz_id_counter += 1
                
                local_quizzes.append({
                    "id": current_quiz_id, 
                    "section": current_section_id, 
                    "quiz_type": quiz.quiz_type, 
                    "question_en": quiz.question_en, 
                    "question_vi": quiz.question_vi, 
                    "audio_url": quiz.audio_url, 
                    "image_url": quiz.image_url, 
                    "explanation_en": quiz.explanation_en, 
                    "explanation_vi": quiz.explanation_vi, 
                    "order": quiz.order
                })

                if quiz.flashcard: local_flashcards.append({"quiz_id": current_quiz_id, **quiz.flashcard.model_dump()})
                if quiz.options: 
                    for opt in quiz.options: local_options.append({"quiz_id": current_quiz_id, **opt.model_dump()})
                if quiz.pairs: 
                    for pair in quiz.pairs: local_pairs.append({"quiz_id": current_quiz_id, **pair.model_dump()})
                if quiz.sentence: local_sentences.append({"quiz_id": current_quiz_id, **quiz.sentence.model_dump()})
                if quiz.speaking: local_speaking.append({"quiz_id": current_quiz_id, **quiz.speaking.model_dump()})
                if quiz.text_response:
                    d = quiz.text_response.model_dump()
                    d['accepted_answers'] = str(d['accepted_answers'])
                    local_text_response.append({"quiz_id": current_quiz_id, **d})

            append_to_csv(OUTPUT_DIR, local_sections, 'sections.csv')
            append_to_csv(OUTPUT_DIR, local_quizzes, 'quizzes.csv')
            append_to_csv(OUTPUT_DIR, local_flashcards, 'flashcards.csv')
            append_to_csv(OUTPUT_DIR, local_options, 'options.csv')
            append_to_csv(OUTPUT_DIR, local_pairs, 'pairs.csv')
            append_to_csv(OUTPUT_DIR, local_sentences, 'sentences.csv')
            append_to_csv(OUTPUT_DIR, local_speaking, 'speaking.csv')
            append_to_csv(OUTPUT_DIR, local_text_response, 'text_response.csv')
            
            print(f"      💾  Saved {len(local_quizzes)} quizzes.")

    print("\n🎉 All Generation Complete!")

def append_to_csv(directory, data_list, filename):
    if not data_list: return
    filepath = os.path.join(directory, filename)
    df = pd.DataFrame(data_list)
    df.to_csv(filepath, mode='a', header=not os.path.isfile(filepath), index=False)

In [84]:
if __name__ == "__main__":
    main()


🚀 Starting Pipeline (Force Save Mode)...


╔══════════════════════════════════════════════════════════════╗
║ Processing Lesson: 你好
╚══════════════════════════════════════════════════════════════╝
   [1/2] 🏗️  Architect is designing...

🔵 [ARCHITECT] Planning structure...


🔎 [ARCHITECT OUTPUT]:
{
  "rationale": "This lesson is designed to introduce students to the basics of Mandarin Chinese, focusing on greetings and the foundational elements of Pinyin and tones. The structure follows a scaffolded approach, starting with vocabulary acquisition, moving through grammar understanding, and culminating in practical application through conversation and speaking exercises. Each section builds on the previous one to ensure comprehensive understanding and retention.",
  "sections": [
    {
      "section_type": "Vocabulary",
      "title_en": "Basic Greetings Vocabulary",
      "title_vi": "Từ vựng Chào hỏi Cơ bản",
      "teaching_goal": "Introduce basic greeting words and phrases.",
      "

KeyboardInterrupt: 